In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 3 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Challenge  interpolation     
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Challenge : interpolation d'une fonction singulière

On cherche à intégrer la fonction $f(x) = \sqrt(x) \log(x)$ sur l'intervalle $]0,1]$.

In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "seaborn"

In [ ]:
def f(x):
    return np.sqrt(x)*np.log(x)

In [ ]:
x = np.linspace(1.e-20, 1, 500)

fig = go.Figure(go.Scatter(x=x, y=f(x), fill='tozeroy', name='f(x)'))
fig.update_layout(title="Intégration de la fonction f(x) = √x log(x)")
fig.show(height=500)

Quelle est la meilleure stratégie à adopter pour intégrer cette fonction en terme de côut pour obtenir une bonne précision (de l'ordre de 1e-13) : monter en ordre pour une formule de quadrature ou fixer un ordre de quadrature et augmenter le nombre de division ? 

Pour répondre à la question précédente, nous avons intégré la fonction $f(x)$ pour différents ordre de la formule de quadrature de Clenshaw-Curtis et pour différents nombres de division.

In [ ]:
def cheb_points(n):
    return np.cos( (2*np.arange(0,n)+1)*np.pi / (2*n) )

def coeffs_clenshawcurtis(n):
    x = cheb_points(n+1) # n+1 Chebyshev nodes
    k = np.arange(0, n+1, dtype='float')
    b = np.zeros(n+1)
    b[0::2] = 2 / (1 - k[0::2]*k[0::2])
    theta = (2*np.arange(0,n+1)+1)/(2*(n+1))*np.pi 
    ## Construction of the matrix B from the notes
    B = np.cos(np.outer(np.arange(0,n+1),theta))/(n+1)
    B[1:,:] = 2*B[1:,:]
    w = np.dot(np.transpose(B), b)
    return x, w

In [ ]:
res_exa = -4./9.

# resultats obtenus avec l'intégration adaptative 
eval_fcn_adapt = 645
err_adapt = 1.9056978218e-13/-res_exa

xmin = 0.
xmax = 1.

nb_intervals = np.array([1, 10, 100, 1000])
nb_pts = np.array([2, 5, 50, 500, 5000])

res = np.zeros((nb_pts.size, nb_intervals.size))
err = np.zeros((nb_pts.size, nb_intervals.size))

for i_n, n in enumerate(nb_pts):
    xk, wk = coeffs_clenshawcurtis(n-1)
    for i_m, m in enumerate(nb_intervals):
        x = np.linspace(xmin, xmax, m+1)
        for j in range(m):
            xminj = x[j]
            xmaxj = x[j+1]
            xkj = xminj + ((xmaxj-xminj)/2)*(xk+1) 
            wkj = ((xmaxj-xminj)/2)*wk
            res[i_n, i_m] += np.sum(wkj*f(xkj))
        err[i_n, i_m] = np.abs(res[i_n, i_m]-res_exa) /  np.abs(res_exa)


fig03 = go.Figure()
for i_n, n in enumerate(nb_pts):
    fig03.add_trace(go.Scatter(x=n*nb_intervals, y=err[i_n], mode="markers", name=f"n={n}"))
fig03.add_trace(go.Scatter(x=[eval_fcn_adapt], y=[err_adapt], mode="markers", name=f"intégration<br>adaptative"))
fig03.update_xaxes(type="log", exponentformat='e', title_text="Nb d'évaluations de fonction")
fig03.update_yaxes(type="log", exponentformat='e', title_text='Erreur')
fig03.update_layout(height=500, title="Erreur en fonction du nb d'évaluations de fcn pour différents ordres de quadrature n")
fig03.show()        
        
fig01 = go.Figure()
for i_m, m in enumerate(nb_intervals):
    fig01.add_trace(go.Scatter(x=nb_pts, y=err[:,i_m], mode="markers", name=f"m={m}"))
fig01.update_xaxes(type="log", exponentformat='e', title_text='Nb de points de quadrature par division')
fig01.update_yaxes(type="log", exponentformat='e', title_text='Erreur')
fig01.update_layout(height=500, title="Erreur en fonction de l'ordre de quadrature pour différents nb de subdivisions m")
fig01.show()

fig02 = go.Figure()
for i_n, n in enumerate(nb_pts):
    fig02.add_trace(go.Scatter(x=nb_intervals, y=err[i_n], mode="markers", name=f"n={n}"))
fig02.update_xaxes(type="log", exponentformat='e', title_text='Nb de subdivisions')
fig02.update_yaxes(type="log", exponentformat='e', title_text='Erreur')
fig02.update_layout(height=500, title="Erreur en fonction du nb de subdivisions pour différents ordres de quadrature n")
fig02.show()

Le lecteur peut observer que la stratégie la plus efficace pour une précision donnée est de prendre un seul intervalle et de profiter de l’excellente stabilité de notre implémentation de la méthode de Clenshaw-Curtis pour monter en nombre de points de quadrature (le point le plus à gauche correspondant à un effort de calcul minimal pour une précision fixée est celui qui correspond à une quadrature élémentaire).
 
Nous sommes dans un contexte où la perte de régularité de la fonction près de zéro met en défaut les théorèmes de convergence que ce soit en termes de nombres d’intervalles de subdivision dans les quadratures composées à nombre de point de quadrature fixé (graphique du milieu) ou à nombre de subdivision donnée en fonction du nombre de points de quadrature (graphique du bas). Par conséquent, le graphique du haut permet de montrer que pour atteindre un précision fixée inférieure à 1.e-13 il faut au moins effectuer une quadrature élémentaire avec 50000 points et si l’on s’en tient à 5000 points de quadrature, alors il faut une centaine d’intervalles, ce qui conduit inéluctablement à un nombre d’évaluation de la fonction de l’ordre de 50 000 à 500 000.
 
Le lecteur pourra observer que pour cette fonction particulière, une quadrature de type adaptative permet de diviser le nombre d’évaluation de la fonction par un facteur de l’ordre de 100, ce qui correspond à une très nette accélération du calcul.

Pour la méthode adaptative, à chaque itération, deux quadratures d'ordre 15 sont calculées, ce qui correspond à 30 évaluations de fonctions. Pour ce cas, cet algorithme a convergé en 21 itérations pour atteindre une précision de l'ordre de 1e-13 (auquels il faut ajouter le calcul de deux quadratures supplémentaires lors de l'initialisation), ce qui fait 21x30 + 15 soit 645 évaluations de fonctions. En toute rigueur, pour évaluer le coût de la méthode adaptative, il faudrait prendre en compte le calcul de l'estimation d'erreur.    